In [1]:

import yaml

import jax
import brax
from brax.io import html

from rl_games.torch_runner import Runner
from rl_games.envs.brax import BraxEnv

from IPython.display import HTML, IFrame, display, clear_output
import os
os.environ["XLA_PYTHON_CLIENT_PREALLOCATE"] = "false"

In [2]:
config_name = 'rl_games/configs/brax/ppo_ant.yaml'
trained_network = 'nn/Ant_brax.pth'

#config_name = 'rl_games/configs/brax/ppo_humanoid.yaml'
#trained_network = 'nn/Humanoid_brax.pth'

In [3]:
with open(config_name, 'r') as stream:
    config = yaml.safe_load(stream)
    runner = Runner()
    runner.load(config)
    config = runner.get_prebuilt_config()
    agent = runner.create_player()
    agent.restore(trained_network)
env_config = config['env_config']

{'observation_space': Box(-inf, inf, (87,), float32), 'action_space': Box(-1.0, 1.0, (8,), float32), 'agents': 1, 'value_size': 1}
build mlp: 87
sigma
actor_mlp.0.weight
actor_mlp.0.bias
actor_mlp.2.weight
actor_mlp.2.bias
actor_mlp.4.weight
actor_mlp.4.bias
value.weight
value.bias
mu.weight
mu.bias
RunningMeanStd:  (87,)
=> loading checkpoint 'nn/Ant_brax.pth'


/home/trrrrr/anaconda3/envs/rlgpu/lib/python3.7/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


In [ ]:
num_actors = 1
env = BraxEnv('', num_actors, **env_config)

In [ ]:
qps = []
obs = env.reset()
total_reward = 0
num_steps = 0


class QP:
    def __init__(self, qp):
        self.pos = jax.numpy.squeeze(qp.pos, axis = 0)
        self.rot = jax.numpy.squeeze(qp.rot, axis = 0)


while not env.state.done:
    qps.append(QP(env.state.qp))
    act = agent.get_action(obs)
    obs, reward, is_done, info = env.step(act.unsqueeze(0))
    total_reward += reward.item()
    num_steps += 1

print('Total Reward: ', total_reward)
print('Num steps: ', num_steps) 


In [ ]:
def visualize(sys, qps):
    return HTML(html.render(sys, qps))

In [ ]:
display(visualize(env.env.sys, qps))